In [41]:
from selenium import webdriver
from selenium.webdriver.firefox.service import Service as FirefoxService
from send2trash import send2trash
from webdriver_manager.firefox import GeckoDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import pandas as pd
import requests
import random
import time
from tqdm import tqdm
from pathlib import Path
import re

# this will be very annoying to do because we'll need to adjust for each publisher

In [22]:
articles = pd.read_csv('/Users/antonhesse/Desktop/Anton/Education/UMN/Lab and Research/HSPL/CPET_scoping_review/data/cpet_articles/unpaywall/unpaywall_info.csv')
ca_articles = articles[articles['is_oa'] == False].reset_index(drop=True)

In [23]:
pdf_paths = list(Path('/Users/antonhesse/Desktop/Anton/Education/UMN/Lab and Research/HSPL/CPET_scoping_review/data/cpet_articles/full_texts/pdfs').rglob('*.pdf'))
pdf_files = [path.stem for path in pdf_paths]
epub_paths = list(Path('/Users/antonhesse/Desktop/Anton/Education/UMN/Lab and Research/HSPL/CPET_scoping_review/data/cpet_articles/full_texts/epubs').rglob('*.epub'))
epub_files = [path.stem for path in epub_paths]
txt_paths = list(Path('/Users/antonhesse/Desktop/Anton/Education/UMN/Lab and Research/HSPL/CPET_scoping_review/data/cpet_articles/full_texts/txts').rglob('*.txt'))
txt_files = [path.stem for path in txt_paths]

In [24]:
full_texts = pdf_files + epub_files + txt_files
full_texts = list(set(full_texts))

In [25]:
re_doi_suffix = re.compile(r'(?<=\d/).*')
ca_articles['doi_suffix'] = ca_articles['doi'].apply(lambda x: re_doi_suffix.search(x).group())

In [28]:
full_texts_to_download = [x for x in ca_articles['doi_suffix'].tolist() if x not in full_texts]
merge = pd.merge(pd.DataFrame({'doi_suffix': full_texts_to_download}), ca_articles, how='inner', on='doi_suffix')

In [77]:
n = random.randint(0, merge.shape[0])
row = merge.loc[n,:]

In [78]:
doi = row['doi']
doi_url = row['doi_url']
print('DOI:', doi)
print('DOI URL:', doi_url)

DOI: 10.1177/2047487320914734
DOI URL: https://doi.org/10.1177/2047487320914734


In [75]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:102.0) Gecko/20100101 Firefox/102.0'}
# dest_folder = 'data/cpet_articles/full_texts/pdfs/aps_non_oa/' # set dest_folder based on publisher

In [71]:
driver = webdriver.Firefox(service=FirefoxService(GeckoDriverManager().install()))
driver.implicitly_wait(1) # hopefully let's JS load correctly

In [79]:
driver.get(url = doi_url)

In [36]:
driver.get(url = 'https://www.lib.umn.edu/') # start at UMN library page

In [63]:
search_bar = driver.find_element(By.XPATH, "//input[@class='form-control form-control-lg' and @name='search']")

In [66]:
search_bar.clear()
search_bar.send_keys(doi)
search_bar.send_keys(Keys.RETURN)